In [15]:
class Matrix:
    
    def __init__(self, *args):

        row_count = 0
        multi_array = []

        for a in args:

            if isinstance(a, list):
                multi_array.append(a)
        
        self.multi_array = multi_array

    def __check_block_match(m1, m2):
        for row in m1:

            try:
                m2[m1.index(e)]
            except ValueError:
                return False

            for e in row:
                try:
                    m2[m1.index(row)][row.index(e)]
                except ValueError:
                    return False
        
        return True


    def __add__(self, other):
        other_matrix = other.multi_array
        my_matrix = self.multi_array
        result = []
        for n in range(len(my_matrix)):
            result.append(list(map(lambda x, y: x + y, my_matrix[n], other_matrix[n])))

        lst = []
        lst = list(map(int, lst))
        lst = (
            int(x) + int(y)
            for x in lst if int(x) == 2 
            for y in lst[::-1]
        )
        return result


In [16]:
my_matrix = [["foo", "bar", "foobar"], [1, 2, 3,]]
other_matrix = [[3, 4, 5,], ["good", "bad", "terrible"]]
c_rows = list(zip(my_matrix, other_matrix))
print(c_rows)
print(c_rows[0][0])

[(['foo', 'bar', 'foobar'], [3, 4, 5]), ([1, 2, 3], ['good', 'bad', 'terrible'])]
['foo', 'bar', 'foobar']


In [19]:
m1 = Matrix([1, 2, 3], [3, 4, 5])
m2 = Matrix([5, 4, 3], [3, 2, 1])
r = m1 + m2
print(r)

[[6, 6, 6], [6, 6, 6]]
